In [3]:
# ===============================================
# 1. Install dependencies
# ===============================================
!pip install pandas replicate --quiet

import pandas as pd
import os, replicate
from getpass import getpass

# ===============================================
# 2. Securely input Replicate token
# ===============================================
if "REPLICATE_API_TOKEN" not in os.environ or not os.environ["REPLICATE_API_TOKEN"].startswith("r8_"):
    os.environ["REPLICATE_API_TOKEN"] = getpass("🔐 Paste your Replicate API token here: ")

if not os.environ["REPLICATE_API_TOKEN"].startswith("r8_"):
    raise ValueError("❌ Invalid or missing token. Get it from https://replicate.com/account")

print("✅ Replicate token stored securely.")


# ===============================================
# 3. Load Dataset (World GDP)
# ===============================================
url = "https://raw.githubusercontent.com/datasets/gdp/master/data/gdp.csv"
df = pd.read_csv(url)
print("✅ Dataset loaded successfully.")
print(df.head())

# Get most recent year for each country
df_latest = df.groupby("Country Name").apply(lambda x: x.loc[x["Year"].idxmax()])
df_latest = df_latest.reset_index(drop=True)
print("\nLatest GDP per Capita Data:\n", df_latest.head())


# ===============================================
# 4. Helper function for Granite calls
# ===============================================
def call_granite(prompt, model="ibm-granite/granite-3.3-8b-instruct", max_tokens=250):
    """Call IBM Granite on Replicate safely, with error handling."""
    try:
        print("\n⏳ Sending prompt to IBM Granite...")
        output = replicate.run(model, input={"prompt": prompt, "max_new_tokens": max_tokens})
        result = "".join(output)
        print("✅ Granite response received.\n")
        return result
    except replicate.exceptions.ReplicateError as e:
        print(f"❌ Granite error: {e}")
        print("Tip: Check your API token or model name.")
        return "Error: Unable to get Granite response."


# ===============================================
# 5. Analysis Prompts
# ===============================================

analysis_prompt = f"""
You are an AI economist analyzing world GDP per capita data.
Here are 10 sample rows (GDP per capita in USD):

{df_latest.head(10).to_string(index=False)}

Tasks:
1. Summarize global GDP trends.
2. Identify 3 key insights about income distribution.
3. Explain how GDP has changed in recent years.
Output as 4-6 clear bullet points.
"""

summary = call_granite(analysis_prompt)
print("🌍 Granite Global Economic Summary:\n", summary)


# ===============================================
# 6. Classification Prompt
# ===============================================
classification_prompt = f"""
Classify the following countries into 3 categories:
- Low Income (< $10,000)
- Middle Income ($10,000–$40,000)
- High Income (> $40,000)

Dataset sample:
{df_latest[['Country Name','Value']].head(10).to_string(index=False)}

Output as a markdown table with columns: Country | Income Category.
"""

classification = call_granite(classification_prompt)
print("📊 Granite Country Classification:\n", classification)


# ===============================================
# 7. Recommendation Prompt
# ===============================================
recommendation_prompt = """
Provide 3 global economic recommendations based on GDP disparities:
1. For low-income countries
2. For middle-income countries
3. For high-income countries
Each with a one-sentence justification.
"""

recommendations = call_granite(recommendation_prompt)
print("💡 Granite Economic Recommendations:\n", recommendations)


# ===============================================
# 8. Save all outputs
# ===============================================
os.makedirs("/content/granite_outputs", exist_ok=True)

with open("/content/granite_outputs/summary.txt", "w") as f: f.write(summary)
with open("/content/granite_outputs/classification.txt", "w") as f: f.write(classification)
with open("/content/granite_outputs/recommendations.txt", "w") as f: f.write(recommendations)

print("\n✅ All Granite outputs saved to /content/granite_outputs/")


✅ Replicate token stored securely.
✅ Dataset loaded successfully.
  Country Name Country Code  Year         Value
0  Afghanistan          AFG  2000  3.521418e+09
1  Afghanistan          AFG  2001  2.813572e+09
2  Afghanistan          AFG  2002  3.825701e+09
3  Afghanistan          AFG  2003  4.520947e+09
4  Afghanistan          AFG  2004  5.224897e+09

Latest GDP per Capita Data:
                   Country Name Country Code  Year         Value
0                  Afghanistan          AFG  2022  1.450216e+10
1  Africa Eastern and Southern          AFE  2023  1.236163e+12
2   Africa Western and Central          AFW  2023  7.965862e+11
3                      Albania          ALB  2023  2.297768e+10
4                      Algeria          DZA  2023  2.398995e+11

⏳ Sending prompt to IBM Granite...


/tmp/ipython-input-1301960052.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_latest = df.groupby("Country Name").apply(lambda x: x.loc[x["Year"].idxmax()])


✅ Granite response received.

🌍 Granite Global Economic Summary:
 1. **Global GDP Trends Summary**:
   - The dataset provided offers a snapshot of GDP per capita for various regions and countries, with years ranging from 2022 to 2023.
   - There is a significant variation in GDP per capita values, reflecting the diverse economic development levels across the world.
   - Some countries, like Afghanistan (AFG) and American Samoa (ASM), have relatively lower GDP per capita, indicating less developed economies.
   - In contrast, regions like the "Africa Eastern and Southern" (AFE) and the "Antarctica" (if it were included, not present here) show considerably higher GDP per capita values, suggesting more developed economies within those regions.

2. **Income Distribution Insights**:
   - **Regional Disparity**: The data highlights substantial income disparities between regions. For instance, the "Africa Eastern and Southern" (AFE) has a GDP per capita around 1.236163e+12 USD, which is signi